In [1]:
from loss import JDAMLoss
from model.sde import VESDE
from model.ddpm import DDPM
from data.data import load_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)
tf.random.set_seed(0)

In [2]:
# unet = DDPM(activation=tf.keras.activations.swish)
# sde = VESDE(
#     pet_score_func=lambda x, t: unet(x, t),
#     mri_score_func=lambda x, t: unet(x, t)
# )

In [3]:
# lk = JDAMLoss(sde)

In [4]:
# data = load_data()

In [5]:
# mri, pet = data[:1, :, :, :1], data[:1, :, :, 1:2]
# mri = tf.cast(mri, dtype=tf.float32)
# pet = tf.cast(pet, dtype=tf.float32)

In [6]:
# losses = []
# for i in range(1, 11):
#     t = tf.convert_to_tensor(i/10, dtype=tf.float32)
#     l = lk.compute_loss_2(unet, pet, mri, t)
#     losses.append((t, l))
#     print("t: ", t.numpy(), "loss: ", l.numpy())

In [7]:
# ts, ls = zip(*losses)
# plt.plot(ts, ls, marker='o')

In [8]:
def plot_images(pet_noisy):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(pet[0, :, :, 0], cmap='gray')
    axes[0].set_title("Original PET")
    axes[0].axis("off")
    
    axes[1].imshow(pet_noisy[0, :, :, 0], cmap='gray')
    axes[1].set_title("Noisy PET")
    axes[1].axis("off")
    
    plt.show()

In [9]:
# _, noise = sde.marginal_probability(pet, t)
# z = tf.random.normal(tf.shape(pet))
# pet_noisy_2 = pet + noise * z
# plot_images(pet_noisy_2)

In [10]:
import train

In [11]:
train.train()

Epoch 1/100: 100%|██████████| 1/1 [00:07<00:00,  7.47s/batch, loss=tf.Tensor(0.9842547, shape=(), dtype=float32)]


Epoch 1/100, Mean Loss: 0.98425, Time: 7.46s


Epoch 2/100: 100%|██████████| 1/1 [00:06<00:00,  6.32s/batch, loss=tf.Tensor(0.9888439, shape=(), dtype=float32)]


Epoch 2/100, Mean Loss: 0.98884, Time: 6.32s


Epoch 3/100: 100%|██████████| 1/1 [00:06<00:00,  6.13s/batch, loss=tf.Tensor(0.99069357, shape=(), dtype=float32)]


...

Epoch 98/100, Mean Loss: 0.99983, Time: 3.49s


Epoch 99/100: 100%|██████████| 1/1 [00:03<00:00,  3.44s/batch, loss=tf.Tensor(1.0027864, shape=(), dtype=float32)]


Epoch 99/100, Mean Loss: 1.00279, Time: 3.44s


Epoch 100/100: 100%|██████████| 1/1 [00:03<00:00,  3.54s/batch, loss=tf.Tensor(0.99201006, shape=(), dtype=float32)]


Epoch 100/100, Mean Loss: 0.99201, Time: 3.54s
